---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [6]:
def answer_one():
    
    
    return spam_data['target'].mean() * 100


In [7]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    X_train_vect = CountVectorizer().fit(X_train)
    
    return max(X_train_vect.get_feature_names(), key=len)

In [19]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer()
    X_train_vect = vect.fit_transform(X_train)
    clf = MultinomialNB(alpha=0.1).fit(X_train_vect, y_train)
    
    X_test_vect = vect.transform(X_test)
    y_predict = clf.predict(X_test_vect)
    
    return roc_auc_score(y_test, y_predict)

In [43]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [221]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    tfidf_index = X_train_vect.max(0).toarray()[0]
    sorted_tfidf_index = tfidf_index.argsort()
    smallesttdifs = pd.Series(tfidf_index[sorted_tfidf_index[:20]], 
                              index=feature_names[sorted_tfidf_index[:20]],
                              name='smallest tf-idfs')
    
    largesttdidfs = pd.Series(tfidf_index[sorted_tfidf_index[-1:-21:-1]], 
                              index=feature_names[sorted_tfidf_index[-1:-21:-1]],
                              name='largest tf-idfs')
    
    return smallesttdifs.shape, largesttdidfs.shape

In [222]:
answer_four()

((20,), (20,))

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [172]:
def answer_five():
    vect = TfidfVectorizer(min_df=3)
    X_train_vect = vect.fit_transform(X_train)
    clf = MultinomialNB(alpha=0.1).fit(X_train_vect, y_train)
    
    X_test_vect = vect.transform(X_test)
    y_predict = clf.predict(X_test_vect)
      
    return roc_auc_score(y_test, y_predict)

In [173]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [113]:
def answer_six():
    spam = spam_data.where(spam_data['target'] == 1).dropna()
    notSpam = spam_data.where(spam_data['target'] == 0).dropna()
    
    return notSpam['text'].str.len().mean(), spam['text'].str.len().mean()

In [114]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [192]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [193]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5)
    X_train_vect = vect.fit_transform(X_train)
    X_train_vect = add_feature(X_train_vect, X_train.str.len())
    clf = SVC(C=10000).fit(X_train_vect, y_train)
    X_test_vect = vect.transform(X_test)
    X_test_vect = add_feature(X_test_vect, X_test.str.len())
    y_predict = clf.predict(X_test_vect)
      
    return roc_auc_score(y_test, y_predict)

In [194]:
answer_seven()

c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [223]:
def answer_eight():
    spam = spam_data.where(spam_data['target'] == 1).dropna()
    notSpam = spam_data.where(spam_data['target'] == 0).dropna()
    aveDigSpam = spam['text'].str.findall(r'[\d\u00BC]').str.len().mean()
    aveDigNotSpam = notSpam['text'].str.findall(r'[\d\u00BC]').str.len().mean()
    
    return aveDigNotSpam, aveDigSpam

In [224]:
answer_eight()

(0.2992746113989637, 15.76037483266399)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [227]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3))
    X_train_vect = vect.fit_transform(X_train)
    X_train_vect = add_feature(X_train_vect, X_train.str.len())
    X_train_vect = add_feature(X_train_vect, X_train.str.findall(r'[\d\u00BC]').str.len())
    clf = LogisticRegression(C=100).fit(X_train_vect, y_train)
    X_test_vect = vect.transform(X_test)
    X_test_vect = add_feature(X_test_vect, X_test.str.len())
    X_test_vect = add_feature(X_test_vect, X_test.str.findall(r'[\d\u00BC]').str.len())
    y_predict = clf.predict(X_test_vect)
      
    return roc_auc_score(y_test, y_predict)

In [228]:
answer_nine()

Empty DataFrame
Columns: [a, b]
Index: []


c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [201]:
def answer_ten():
    spam = spam_data.where(spam_data['target'] == 1).dropna()
    notSpam = spam_data.where(spam_data['target'] == 0).dropna()
    aveNonWordSpam = spam['text'].str.findall(r'\W').str.len().mean()
    aveNonWordNotSpam = notSpam['text'].str.findall(r'\W').str.len().mean()
    
    return aveNonWordNotSpam, aveNonWordSpam 

In [202]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [253]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb')
    X_train_vect = vect.fit_transform(X_train)
    X_train_vect = add_feature(X_train_vect, X_train.str.len())
    X_train_vect = add_feature(X_train_vect, X_train.str.findall(r'[\d\u00BC]').str.len())
    X_train_vect = add_feature(X_train_vect, X_train.str.findall(r'\W').str.len())
    
    clf = LogisticRegression(C=100).fit(X_train_vect, y_train)
    
    X_test_vect = vect.transform(X_test)
    X_test_vect = add_feature(X_test_vect, X_test.str.len())
    X_test_vect = add_feature(X_test_vect, X_test.str.findall(r'[\d\u00BC]').str.len())
    X_test_vect = add_feature(X_test_vect, X_test.str.findall(r'\W').str.len())
    y_predict = clf.predict(X_test_vect)
    
    feature_names = np.array(vect.get_feature_names() 
                             + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    sorted_coef_index = clf.coef_[0].argsort()
    small_coef = feature_names[sorted_coef_index[:10]].tolist()
    large_coef = feature_names[sorted_coef_index[-1:-11:-1]].tolist()
    
    return roc_auc_score(y_test, y_predict), small_coef, large_coef

In [254]:
answer_eleven()

c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.9788593110707434,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])